In [ ]:
!pip install -q git+https://github.com/tensorflow/docs
!pip install -q scikit-learn

In [ ]:
import io
import sklearn
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_docs.plots
import tensorflow_docs.modeling
import tensorflow_docs as tfdocs
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers

warnings.filterwarnings('ignore')

print('Tensorflow version: ', tf.__version__)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
column_names = ['yield', 'yield-1', 'prcp10', 'prcp11', 'prcp12', 'prcp01', 'prcp02', 'avgtemp', 'maxtemp', 'mintemp']

data = pd.read_csv(io.BytesIO(uploaded['Dataset thesis.csv']), sep=';')

dataset = data.copy()
data.tail()

Splitting the data into train and test

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

print (train_dataset.shape)
print (test_dataset.shape)

Inspecting the data

In [ ]:
##%matplotlib inline

In [ ]:
##sns.pairplot(train_dataset[['yield', 'yield-1', 'prcp10', 'avgtemp']], diag_kind='kde')

In [ ]:
##sns.pairplot(data, x_vars=['yield-1', 'prcp10', 'prcp11', 'prcp12', 'prcp01', 'prcp02', 'avgtemp', 'maxtemp', 'mintemp'], y_vars=['yield'], size=7, aspect=0.7, kind='reg') 

Splitting features from labels

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('yield')
test_labels = test_features.pop('yield')

Normalization using Min-Max Scaling

In [ ]:
normalized_train = (train_dataset - train_dataset.min(axis=0)) / (train_dataset.max(axis=0) - train_dataset.min(axis=0))
normalized_test = (test_dataset - test_dataset.min(axis=0)) / (test_dataset.max(axis=0) - test_dataset.min(axis=0))
normalized_train_labels = (train_labels - train_labels.min(axis=0)) / (train_labels.max(axis=0) - train_labels.min(axis=0))
normalized_test_labels = (test_labels - test_labels.min(axis=0)) / (test_labels.max(axis=0) - test_labels.min(axis=0))

In [ ]:
def build_model():
  model = keras.Sequential([
      layers.Dense(9, activation = tf.nn.relu),
      layers.Dense(5, activation = tf.nn.relu),
      layers.Dense(1)
  ])
  
  optimizer = tf.keras.optimizers.SGD(0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=[tf.keras.metrics.RootMeanSquaredError,'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

epochs = 1000

history = model.fit(
    normalized_train, train_labels,
    validation_split=0.2, verbose=0, epochs=epochs,
    callbacks=[PrintDot()])

In [ ]:
plot_loss(history)